# Data

In [ ]:
azureADExport = """"""

# System Prerequisists and consts

In [ ]:
# install slack sdk
!pip install slack-sdk

In [ ]:
csvHeader = ['slackScoutname', 'slackRealname', 'slackEmail', 'slackTitle', 'PictureURL', 'adEmail', 'adTitle', 'adSector']
exportPath = "/content/SlackExport/"
csvFileName = "slackprofilepictureexport.csv"

# Main

In [ ]:
class User:
  def __init__(self, slackScoutname, slackRealname, slackEmail, slackTitle, pictureURL, adEmail, adTitle, adSector):
    self.slackScoutname = slackScoutname
    self.slackRealname = slackRealname
    self.slackEmail = slackEmail
    self.slackTitle = slackTitle
    self.pictureURL = pictureURL

    self.adEmail = adEmail
    self.adTitle = adTitle
    self.adSector = adSector

  def __repr__(self):
    return "slackScoutname: " + self.slackScoutname + ", slackRealname: " + self.slackRealname + ", slackEmail: " + self.slackEmail.lower() + ", Title: " + self.slackTitle + ", PictureURL: " + self.pictureURL + ", adEmail: " + self.adEmail + ", adTitle: " + self.adTitle + ", adSector: " + self.adSector

  def __iter__(self):
    return iter((self.slackScoutname, self.slackRealname, self.slackEmail, self.slackTitle, self.pictureURL, self.adEmail, self.adTitle, self.adSector))

  def get_image_name(self):
    return self.pictureURL.split("/")[4]

  def download_image(self):
    import urllib.request

    if self.pictureURL:
      fileEnding = self.get_image_name().split(".")[1]
      urllib.request.urlretrieve(self.pictureURL, exportPath + self.slackEmail + "." + fileEnding)
    else:
      print("No picture for " + str(self))

def load_azure_ad():
  # read data file and skip first line
  lines = azureADExport.splitlines()
  reader = csv.reader(lines, delimiter=',')
  next(reader)

  return reader

class AzureADUser: 
  def __init__(self, email, title, sector):
    self.email = email
    self.title = title
    self.sector = sector

  def from_data(row):
    return AzureADUser(email = row[8].lower(), title=row[7], sector=row[0])

  def __repr__(self):
    return "Email: " + str(self.email) + ", Title: " + str(self.title) + ", Sector: " + str(self.sector)

In [ ]:
# Import WebClient from Python SDK
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import csv

# WebClient instantiates a client
client = WebClient(token="")

# Azure AD Users
azureADUsers = {}
azureADUsers = { user.email: user for user in list(map(lambda data: AzureADUser.from_data(data), load_azure_ad())) }

In [ ]:
# get all users
result = client.users_list()
result = result["members"]

In [ ]:
# get james
testAccount = list(filter(lambda user: "profile" in user.keys() and "email" in user["profile"].keys() and user["profile"]["email"] == "james.levell@bula21.ch", result))

In [ ]:
users = []    # output array
if len(result) > 0:
  for user in result:
    if not user["is_bot"] and not user["deleted"]:
      slackEmail = ""
      slackScoutname = user["name"]
      slackRealname = ""
      slackTitle = ""
      pictureURL = ""

      adEmail = ""
      adTitle = ""
      adSector = ""

      if "profile" in user.keys():
        if "display_name" in user["profile"].keys():
          slackScoutname = user["profile"]["display_name"]

        if "image_original" in user["profile"].keys():
          pictureURL = user["profile"]["image_original"]

        if "real_name" in user["profile"].keys():
          slackRealname = user["profile"]["real_name"]

        if "email" in user["profile"].keys():
          slackEmail = user["profile"]["email"]
          slackEmail = slackEmail.lower()

          if slackEmail in azureADUsers.keys():
              adEmail = azureADUsers[slackEmail].email
              adTitle = azureADUsers[slackEmail].title
              adSector = azureADUsers[slackEmail].sector

        if "title" in user["profile"].keys():
          title = user["profile"]["title"]

      slackUser = User(slackScoutname=slackScoutname, slackRealname=slackRealname, slackEmail=slackEmail, slackTitle=slackTitle, pictureURL=pictureURL, adEmail=adEmail, adTitle=adTitle, adSector=adSector)
      slackUser.download_image()

      users.append(slackUser)
else:
  print("No users have been loaded")

In [ ]:
import csv  

header = csvHeader

with open(exportPath + csvFileName, 'w', encoding='UTF8') as csvFile:
  writer = csv.writer(csvFile)
  writer.writerow(header)
  writer.writerows(users)